# Práctica 10: AEs

En esta práctica usaremos un autoencoder como detector de anomalías. Para ello entrenaremos un autoenconder con un conjunto de imágenes de radiografias de tórax de individuos sanos y consideraremos como anomalías las radiografías de individuos con neumonía

In [17]:
!pip install torch numpy matplotlib scikit-image numba cython setuptools medmnist

In [18]:
!pip install numpy --force-reinstall

  Using cached numpy-2.2.5-cp311-cp311-macosx_14_0_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-macosx_14_0_x86_64.whl (7.0 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain 0.3.15 requires SQLAlchemy<3,>=1.4, which is not installed.
langchain-community 0.3.15 requires SQLAlchemy<3,>=1.4, which is not installed.
langchain 0.3.15 requires numpy<2,>=1.22.4; python_version < "3.12", but you have numpy 2.2.5 which is incompatible.
langchain-community 0.3.15 requires numpy<2,>=1.22.4; python_version < "3.12", but you have numpy 2.2.5 which is incompatible.
tensorflow-macos 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 2.2.5 which is incompatible.
langchain-chroma 0.2.0 requires num

In [19]:
!pip uninstall torch torchvision torchaudio -y
!pip install torch torchvision torchaudio


Found existing installation: torch 2.2.2
Uninstalling torch-2.2.2:
  Successfully uninstalled torch-2.2.2
Found existing installation: torchvision 0.17.2
Uninstalling torchvision-0.17.2:
  Successfully uninstalled torchvision-0.17.2
  Using cached torchvision-0.17.2-cp311-cp311-macosx_10_13_x86_64.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.8/150.8 MB 26.3 MB/s eta 0:00:0000:0100:01
Using cached torchvision-0.17.2-cp311-cp311-macosx_10_13_x86_64.whl (1.7 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 27.5 MB/s eta 0:00:00


In [20]:
import numpy as np
print(np.__version__)

2.2.5


## Dataset

Usaremos un subconjunto de [__MedMNIST__](https://medmnist.com/): un dataset con diversos conjuntos de datos de imágenes médicas. En concreto usaremos __PneumoniaMNIST__, que tiene las siguientes características:
- Más de 5000 imágenes divididas en conjuntos de training, validation y testing.
- Imágenes monocromas de 28x28 píxeles
- Directamente utilizable desde pytorch
- Cada imagen tiene una etiqueta asociada que indica si la imagen es de un individuo sano o con neumonía.

Para poder utilizar el dataset, primero deberemos instalar el paquete medmnist: ``pip install medmnist``

A continuación se incluye un código python que permite un acceso básico al dataset y muestra mosaicos de imágenes de individuos sanos y con neumonía.

In [21]:
import medmnist
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

print(medmnist.__version__)
from medmnist import PneumoniaMNIST

3.0.2


In [22]:
# Información del dataset
download = True
info = medmnist.INFO['pneumoniamnist']
print(info)

# Obtener la clase de datos
DataClass = getattr(medmnist, info['python_class'])

# Transformaciones de imagen
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Cargar dataset
train_dataset = DataClass(split='train', transform=transform, download=download)

healthy_images = []
pneumonia_images = []

for img, label in train_dataset:
    if label.item() == 0:
        healthy_images.append(img.squeeze())  # quitar dimensión del canal
    if len(healthy_images) == 12:
        break

for img, label in train_dataset:
    if label.item() == 1:
        pneumonia_images.append(img.squeeze())
    if len(pneumonia_images) == 12:
        break

{'python_class': 'PneumoniaMNIST', 'description': 'The PneumoniaMNIST is based on a prior dataset of 5,856 pediatric chest X-Ray images. The task is binary-class classification of pneumonia against normal. We split the source training set with a ratio of 9:1 into training and validation set and use its source validation set as the test set. The source images are gray-scale, and their sizes are (384−2,916)×(127−2,713). We center-crop the images and resize them into 1×28×28.', 'url': 'https://zenodo.org/records/10519652/files/pneumoniamnist.npz?download=1', 'MD5': '28209eda62fecd6e6a2d98b1501bb15f', 'url_64': 'https://zenodo.org/records/10519652/files/pneumoniamnist_64.npz?download=1', 'MD5_64': '8f4eceb4ccffa70c672198ea285246c6', 'url_128': 'https://zenodo.org/records/10519652/files/pneumoniamnist_128.npz?download=1', 'MD5_128': '05b46931834c231683c68f40c47b2971', 'url_224': 'https://zenodo.org/records/10519652/files/pneumoniamnist_224.npz?download=1', 'MD5_224': 'd6a3c71de1b945ea11211b

RuntimeError: Numpy is not available

In [ ]:
# Crear mosaico de imágenes sanas
fig, axes = plt.subplots(3, 4, figsize=(5, 4))
fig.suptitle("Healthy (Label = 0) Images from PneumoniaMNIST", fontsize=16)
for i, ax in enumerate(axes.flat):
    ax.imshow(healthy_images[i], cmap='gray')
    ax.axis('off')
plt.tight_layout()
plt.show()

Obsérvese que el dataset estará compuesto de todas las imágenes, y que para distinguirlas habrá que hacer uso de la etiqueta asociada a cada imagen.

## Modelo

Se probarán las prestaciones de modelos autoencoders estándar (AE) y variacionales (VAE). Como punto de partida se propone la estructura propuesta en el paper disponible en el siguiente [enlace](https://essay.utwente.nl/98159/1/Reve_BA_EEMCS.pdf)

Obsérvese en la fig. 6 del paper la estructura del autoencoder utilizado, en la cual el _bottleneck_ es de tamaño 4x4x8 = 128 sin aplanar

## Tareas a realizar en la práctica

1. Estudiar las características del dataset y plantear un marco experimental adecuado: entrenar con las imágenes del conjunto de training de individuos sanos y usar un conjunto de validación y otro de test balanceados (mismo número de imágenes de individuos sanos y con neumonía)
2. Sobre el conjunto de validación, hacer un estudio de la distribución de los errores (media y desviación de los errores, histograma o nube de puntos como en la fig. 5 del paper) y obtener el umbral que minimiza el error de reconocimiento
3. Con este umbral, calcular las prestaciones del detector de anomalías sobre el conjunto de test. Calcular también probabilidad de falsos positivos y de falsos negativos.